In [1]:
import numpy as np

In [2]:
# 将数组中元素拼接组合
def array_to_string(array, elem_bit):
    val = 0
    for i in range(len(array)):
        tmp = array[i]
        tmp2 = tmp
        
        if tmp < 0:
            tmp2 = 2**(elem_bit) + tmp
            
        tmp2 = int(tmp2)
        tmp3 = tmp2 * 2**(elem_bit*i)
        val = val + tmp3
    return val

In [3]:
class ParamProcess:
    def __init__(self, name, config, w, bias, w_bit, in_bit, out_bit, bias_bit, pe, simd, r_shift):
        self.name = name
        self.config = config
        self.w = w
        self.bias = bias
        self.w_bit = w_bit
        self.in_bit = in_bit
        self.out_bit = out_bit
        self.bias_bit = bias_bit
        self.pe = pe
        self.simd = simd
        self.r_shift = r_shift
    
    # 将矩阵整理成所需的存储样式
    # 转化为pe * tiles矩阵
    def w_to_hls_array(self, w):
        # print(#"w shape: ", w.shape)
        assert w.shape[0] % self.pe == 0, 'out_ch mod pe must 0'
        # w 矩阵宽 k*K*in_ch
        h = w.shape[1]
        # res0 size = out_ch, k*K*in_ch // simd + (0 or 1)
        res0 = [[0 for i in range(h // self.simd)] for j in range(w.shape[0])]
        for out_ch in range(w.shape[0]):
            for i in range(h // self.simd):
                arr = w[out_ch][i*self.simd:(i+1)*self.simd]
                res0[out_ch][i] = array_to_string(arr, self.w_bit)
        
        # 处理不够整除部分
        if h % self.simd != 0:
            print('h mod simd != 0')
            for out_ch in range(w.shape[0]):
                arr = w[out_ch][h // self.simd * self.simd]
                res0[out_ch].append(array_to_string(arr, self.w_bit))
                
        
        tiles = len(res0[0]) * (len(res0) // self.pe)
        self.w_tiles = tiles
        # print('tiles', tiles)
        res = [[0 for i in range(tiles)] for i in range(self.pe)]
        
        tiles_cnt = 0
        for i in range(len(res0) // self.pe):
            for j in range(len(res0[0])):
                for pe_cnt in range(self.pe):
                    res[pe_cnt][tiles_cnt] = res0[i * self.pe + pe_cnt][j]
                tiles_cnt += 1
        return res
    
    def bias_to_hls_array(self, bias):
        bias = bias.reshape(-1, self.pe)
        bias = bias.T
        
        return bias
    
    def conv(self):
        w = self.w
        bias = self.bias
        # w是二维矩阵形式
        conv_w = w.transpose(0, 2, 3, 1)
        # 处理为二维矩阵
        conv_w = conv_w.reshape(conv_w.shape[0], -1)
        # print(w.shape)
        # 先把w处理为每个元素位宽都是simd * w_bit形式
        conv_w = self.w_to_hls_array(conv_w)
        
        bias = self.bias_to_hls_array(bias)
        
        self.hls_w = conv_w
        self.hls_bias = bias
        
        return conv_w, bias
    
    def w_to_hls_init_str(self, w) -> str:
        w_mem_type = "const ap_uint<"+str(self.w_bit * self.simd)+">"
        
        res = '//'  + self.name + '_w\n'
        res += '//PEs = %d, SIMD = %d\n' % (self.pe, self.simd)
        res += '//bit = %d\n' % self.w_bit
        res += w_mem_type
        res += (' ' + self.name + '_w')
        res += '[%d][%d] = {\n' % (len(w), len(w[0]))
        
        res += ",\n".join(map(lambda pe:"{\""+("\", \"".join(map(hex, pe)))+"\"}", w))
        res += '};\n'
        
        return res
    
    def bias_to_hls_init_str(self, bias) -> str:
        bias_bit_width = self.bias_bit
        
        w_mem_type = "const ap_int<"+str(self.bias_bit)+">"
        
        res = '// bias\n'
        res += '//'  + self.name + '_bias\n'
        res += '//w_bit = %d\n' % bias_bit_width
        res += w_mem_type
        res += (' ' + self.name + '_bias')
        res += '[%d][%d] = {\n' % (len(bias), len(bias[0]))
        
        res += ",\n".join(map(lambda pe:"{\""+("\", \"".join(map(hex, pe)))+"\"}", bias))
        res += '};\n'
        
        return res
    
    def layer_param_to_init_str(self, w, bias) -> str:
        res = self.w_to_hls_init_str(w)
        res += self.bias_to_hls_init_str(bias)
        
        return res
    
    def add_a_config_str(self, config_name, value) -> str:
        res = '#define %s_%s %d \n' % (self.name.upper(), config_name.upper(), value)
        return res

    def conv_config_str(self) -> str:
        res = '// ' + self.name + '\n'
        res += self.add_a_config_str('K', self.config['k'])
        res += self.add_a_config_str('S', self.config['s'])
        res += self.add_a_config_str('P', self.config['p'])
        res += self.add_a_config_str('IFM_CH', self.config['in_shape'][0])
        res += self.add_a_config_str('IFM_ROW', self.config['in_shape'][1])
        res += self.add_a_config_str('IFM_COL', self.config['in_shape'][2])

        res += self.add_a_config_str('OFM_CH', self.config['out_shape'][0])
        res += self.add_a_config_str('OFM_ROW', self.config['out_shape'][1])
        res += self.add_a_config_str('OFM_COL', self.config['out_shape'][2])

        res += self.add_a_config_str('SIMD', self.simd)
        res += self.add_a_config_str('PE', self.pe)

        res += self.add_a_config_str('IN_BIT', self.in_bit)
        res += self.add_a_config_str('OUT_BIT', self.out_bit)
        res += self.add_a_config_str('W_BIT', self.w_bit)
        res += self.add_a_config_str('BIAS_BIT', self.bias_bit)

        res += self.add_a_config_str('R_SHIFT', self.r_shift)

        res += '\n'

        return res

In [4]:
def model_analysis(code, pe, simd, r_shift):
    config_dic = {}
    
    channel_num = np.array(code[:5])*np.array([64, 64, 128, 256, 512])
    #print(channel_num)
    layer_num = code[-5:]
    #print(layer_num)
    map_pool = {0:True,1:True,2:False}
    pools = [map_pool[key] for key in code[10:16]] 
    #print(pools)
    shape_size = [32,32,16,8,4,2]
    
    # initial channel
    in_channel = 3
    out_channel = int(channel_num[0])
    
    param_num = 0
    
    for i in range(5):
        for j in range(layer_num[i]):
            name = "conv_"+str(i)+"_"+str(j)
            # initial config
            config = {'k':3, 's':1, 'p':1, 'in_shape':(3, 32, 32), 'out_shape':(8, 32, 32)}
            
            # modify 'in_shape'
            config['in_shape'] = (in_channel, shape_size[i], shape_size[i])
            
            if (j == layer_num[i]-1):
                if (not pools[i]) :
                    # no pooling 
                    config['s'] = 2
                    config['out_shape'] = (out_channel, shape_size[i+1], shape_size[i+1])
                    config['simd'] = simd[param_num]
                    config['pe'] = pe[param_num]
                    config['r_shift'] = r_shift[param_num]
                    config_dic[name]=config

                else :
                    # pooling
                    config['s'] = 1
                    config['out_shape'] = (out_channel, shape_size[i], shape_size[i])
                    config['simd'] = simd[param_num]
                    config['pe'] = pe[param_num]
                    config['r_shift'] = r_shift[param_num]
                    config_dic[name]=config
                    
                    pool_name = "pool_"+str(i)
                    config_dic[pool_name]={'k':2, 's':1, 'p':0}

                # renew channel
                if i < 4 :
                    in_channel = out_channel
                    out_channel = int(channel_num[i+1])
                
                param_num += 1
                
            else :
                config['s'] = 1
                config['out_shape'] = (out_channel, shape_size[i], shape_size[i])
                config['simd'] = simd[param_num]
                config['pe'] = pe[param_num]
                config['r_shift'] = r_shift[param_num]
                config_dic[name]=config

                # renew channel
                in_channel = out_channel
                out_channel = int(channel_num[i])
                
                param_num += 1
            
    return config_dic

In [5]:
def config_param(code, config_dic):
    channel_num = np.array(code[:5])*np.array([64, 64, 128, 256, 512])
    layer_num = code[-5:]
    
    conv = ""
    conv_config = ""
    
    for i in range(5):
        for j in range(layer_num[i]):
            name = "conv_"+str(i)+"_"+str(j)
            config = config_dic[name]
            
            weight_shape = (int(config['out_shape'][0]), int(config['in_shape'][0]), 3, 3)
            weight = np.random.randint(-26, 256, size=weight_shape)
            #print(weight.shape)
            
            bias_shape = (int(config['out_shape'][0]))
            bias = np.random.randint(-256, 255, size=bias_shape)
            
            processer = ParamProcess(
                name=name, 
                config=config,
                w=weight, bias=bias, 
                w_bit=8, in_bit=8, out_bit=8, bias_bit=32, 
                pe=config['pe'], simd=config['simd'], r_shift=config['r_shift'])
            
            w_str, b_str = processer.conv()
            conv_str = processer.layer_param_to_init_str(w_str, b_str)
            conv_config_str = processer.conv_config_str()
            
            conv += conv_str
            conv_config += conv_config_str
    
    return conv, conv_config

In [6]:
code = [1.0, 1.0, 0.25, 0.375, 0.75, 0, 1, 0, 1, 1, 0, 1, 0, 1, 2, 0, 1, 3, 2, 5, 1]
pe = [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
simd = [3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
r_shift = [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

config = model_analysis(code, pe, simd, r_shift)
conv_param, conv_config = config_param(code, config)

hls_param_file = open('param.h', 'w')
hls_config_file = open('config.h', 'w')

hls_param_file.write(conv_param)
hls_config_file.write(conv_config)

hls_param_file.close()
hls_config_file.close()